In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate, KFold

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve

from sklearn.metrics import accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix


from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import pickle
from sklearn.metrics import ConfusionMatrixDisplay
import warnings
warnings.filterwarnings('ignore')
%load_ext autotime

from aux_func import evaluate_model

time: 0 ns (started: 2021-12-12 22:56:13 +01:00)


In [4]:
xtrain = pd.read_parquet("../data/xtrain.parquet")
ytrain = pd.read_parquet("../data/ytrain.parquet")['fatality']
xtest = pd.read_parquet("../data/xtest.parquet")
ytest = pd.read_parquet("../data/ytest.parquet")['fatality']
xtrain_smote = pd.read_parquet("../data/xtrain_smote.parquet")
ytrain_smote = pd.read_parquet("../data/ytrain_smote.parquet")['fatality']

time: 813 ms (started: 2021-12-12 22:56:17 +01:00)


# Random Forest

Estos modelos surgen como una alternativa a los árboles de decisión, y el problema que tienen cuando se usan de manera individual. Al utilizar un árbol de decisión, los resultados no son consistentes, y pueden variar en función de cambios en la composición de los conjuntos de entrenamiento y validación.

El Random Forest utiliza muchos árboles, y la decisión final sobre una predicción concreta se basa en un sistema de voto. Este sistema se puede modificar, para ponderar el peso de los votos de los árboles en función de, por ejemplo, su precisión o el ROC-AUC Score (Área debajo de la curva ROC)

Cada árbol realiza su partición en conjunto de train y test, y se ejecuta. No hay ningún tipo de regla que influya en qué observaciones puede usar un algortimo, son independientes de las que haya utilizado otro arbol

Procedemos a entrenar el modelo.

![Highway](https://www.freecodecamp.org/news/content/images/2020/08/how-random-forest-classifier-work.PNG)

In [48]:
clf = RandomForestClassifier(n_jobs=-1, random_state=0)
clf.fit(xtrain, ytrain)

RandomForestClassifier(n_jobs=-1, random_state=0)

time: 4min 20s (started: 2021-12-12 22:09:45 +01:00)


In [49]:
with open('../models/RandomForest.pickle', 'wb') as f:
    pickle.dump(clf, f)

time: 1.75 s (started: 2021-12-12 22:14:06 +01:00)


In [5]:
# Para no tener que ejecutar, saltarse el fit y ejecutar a partir de aquí
with open('../models/RandomForest.pickle', 'rb') as f:
    clf = pickle.load(f)

time: 3.16 s (started: 2021-12-12 22:56:21 +01:00)


Generamos las predicciones sobre los datos de validación y evaluamos el modelo.

In [6]:
ypred = clf.predict(xtest)
ypred_proba = clf.predict_proba(xtest)
evaluate_model(ytest,ypred,ypred_proba)

ROC-AUC score of the model: 0.8150581847185245
Accuracy of the model: 0.9847850076901997

Classification report: 
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    797650
           1       0.63      0.03      0.06     12472

    accuracy                           0.98    810122
   macro avg       0.81      0.51      0.52    810122
weighted avg       0.98      0.98      0.98    810122


time: 14.7 s (started: 2021-12-12 22:56:26 +01:00)


## Ajuste del umbral de predicción

Al observar cómo nuestro modelo obtiene un valor de recall muy bajo para la clase minoritaria, nos planteamos ajustar el umbral de predicción siguiendo el punto que obtiene la mayor G-mean o media geométrica entre las tasas de "true positives" y "false positives". La curva ROC nos permite determinar este punto, como observaremos más adelante.

In [7]:
# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytest,ypred_new_threshold,ypred_proba)

Best Threshold=0.017500, G-Mean=0.753
ROC-AUC score of the model: 0.8150581847185245
Accuracy of the model: 0.7658821757710567

Classification report: 
              precision    recall  f1-score   support

           0       0.99      0.77      0.87    797650
           1       0.05      0.74      0.09     12472

    accuracy                           0.77    810122
   macro avg       0.52      0.75      0.48    810122
weighted avg       0.98      0.77      0.85    810122


time: 1.41 s (started: 2021-12-12 22:56:41 +01:00)


Podemos observar como el ajuste del threshold dota al modelo de un mayor recall para los casos de la clase minoritaria, lo cual nos interesa desde un punto de vista práctico a pesar de reducir la precisión y accuracy del modelo. 

## Comprobación de overfitting

Comprobamos si el modelo sufre de overfitting, realizando una predicción sobre la serie de entrenamiento.

In [8]:
ypred = clf.predict(xtrain)
ypred_proba = clf.predict_proba(xtrain)

# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytrain, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytrain,ypred_new_threshold,ypred_proba)

Best Threshold=0.257488, G-Mean=1.000
ROC-AUC score of the model: 0.999998720920466
Accuracy of the model: 0.9998475538252263

Classification report: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   3192113
           1       0.99      1.00      0.99     48375

    accuracy                           1.00   3240488
   macro avg       1.00      1.00      1.00   3240488
weighted avg       1.00      1.00      1.00   3240488


time: 56.3 s (started: 2021-12-12 22:56:42 +01:00)


En efecto, el model Random Forest ha realizado un ajuste bastante notable. Posteriormente profundizaremos en este asunto, pero podemos afirmar que el modelo no generaliza bien.

# Random-Forest con SMOTE

En los modelos Random Forest y XGBoost también hemos realizado una prueba con una versión de los datos en los que realizamos un Oversample de la clase minoritaria mediante el algoritmo SMOTE. Probaremos esta técnica en estos primeros dos modelos más complejos con el objetivo de ver si merece la pena. Si consideramos que los resultados son peores a los obtenidos con los datos sin Oversample, nos mantendremos en la senda de usar los datos sin Oversample.

In [53]:
clf = RandomForestClassifier(n_jobs=-1, random_state=0)
clf.fit(xtrain_smote, ytrain_smote)

RandomForestClassifier(n_jobs=-1, random_state=0)

time: 5min 37s (started: 2021-12-12 22:14:21 +01:00)


In [54]:
with open('../models/RandomForest_smote.pickle', 'wb') as f:
    pickle.dump(clf, f)

time: 1.81 s (started: 2021-12-12 22:19:58 +01:00)


In [55]:
# Para no tener que ejecutar, saltarse el fit y ejecutar a partir de aquí
with open('../models/RandomForest_smote.pickle', 'rb') as f:
    clf = pickle.load(f)

time: 4.25 s (started: 2021-12-12 22:20:00 +01:00)


In [56]:
ypred = clf.predict(xtest)
ypred_proba = clf.predict_proba(xtest)
evaluate_model(ytest,ypred,ypred_proba)

ROC-AUC score of the model: 0.8180842229199814
Accuracy of the model: 0.984680085221732

Classification report: 
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    797650
           1       0.53      0.04      0.08     12472

    accuracy                           0.98    810122
   macro avg       0.76      0.52      0.54    810122
weighted avg       0.98      0.98      0.98    810122


time: 15.1 s (started: 2021-12-12 22:20:05 +01:00)


El modelo sin aplicar el ajuste de threshold no es lo suficientemente descriptivo como para poder compararlo con su versión análoga. Procedemos a realizar dicho ajuste.

## Ajuste del umbral de predicción

In [57]:
# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytest,ypred_new_threshold,ypred_proba)


Best Threshold=0.022500, G-Mean=0.751
ROC-AUC score of the model: 0.8180842229199814
Accuracy of the model: 0.7873332658537849

Classification report: 
              precision    recall  f1-score   support

           0       0.99      0.79      0.88    797650
           1       0.05      0.72      0.09     12472

    accuracy                           0.79    810122
   macro avg       0.52      0.75      0.49    810122
weighted avg       0.98      0.79      0.87    810122


time: 1.38 s (started: 2021-12-12 22:20:20 +01:00)


Tras realizar el ajuste del modelo, podemos afirmar que en nuestro caso concreto y para el tratamiento de los datos que hemos realizado, no existe una razón por la que usar la versión Oversample de los datos.
Obtenemos una ligera diferencia en los resultados, obteniendo menor recall para la clase minoritaria pero mayor para la clase mayoritaria. Comprobaremos si ocurre lo mismo en el modelo XGBoost.